In [2]:
pip install pytsk3


  Using cached pytsk3-20231007.tar.gz (3.1 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build pytsk3
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for pytsk3 (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      running bdist_wheel
      running build
      running build_ext
      building 'pytsk3' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytsk3
ERROR: Could not build wheels for pytsk3, which is required to install pyproject.toml-based projects

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import pytsk3


ModuleNotFoundError: No module named 'pytsk3'

In [ ]:
# Define the disk image path
image_path = "path_to_disk_image.dd"  # Replace with your disk image file

# Output directory for recovered files
output_dir = "recovered_files"
os.makedirs(output_dir, exist_ok=True)


In [ ]:
def recover_deleted_files(partition_addr, output_dir):
    img_info = pytsk3.Img_Info(image_path)
    fs_info = pytsk3.FS_Info(img_info, addr=partition_addr)

    for directory_entry in fs_info.open_dir(path="/", addr=partition_addr):
        for file_entry in directory_entry:
            if file_entry.info.name.type == pytsk3.TSK_FS_NAME_TYPE_UNDEF:
                continue
            try:
                file_content = fs_info.open_meta(file_entry.info)
                if file_content.info.meta.flags == pytsk3.TSK_FS_META_FLAG_ALLOC:
                    with open(os.path.join(output_dir, file_entry.info.name.name), "wb") as f:
                        f.write(file_content.read_random(
                            0, file_entry.info.meta.size))
            except Exception as e:
                print(
                    f"Error recovering {file_entry.info.name.name}: {str(e)}")


In [ ]:
if __name__ == "__main__":
    partition_table = pytsk3.Volume_Info(image_path)
    for partition in partition_table:
        if len(partition) > 0:
            print(f"Recovering deleted files from partition {partition.addr}")
            recover_deleted_files(partition.addr, output_dir)

print("File recovery complete. Recovered files are in the 'recovered_files' directory.")
